<a href="https://colab.research.google.com/github/Vamshi-B558/ATHARVO--Machine_Learning/blob/main/IEEE_research_paper_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')

import os
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import  stopwords
import string
import spacy
import pprint

from gensim.models import LdaModel, CoherenceModel
from gensim.corpora import Dictionary

!pip install pyLDAvis
import pyLDAvis.gensim
import pickle
import pyLDAvis

SEED = 7



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_raw = pd.read_csv('/content/database.csv')
data_raw.drop_duplicates(subset = ['title'], inplace = True)
data_raw = data_raw.sample(frac = 1, random_state = SEED).reset_index(drop = True)
data_raw.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Id,title,link,year,authors,citations,abstract
0,261,261,Interplay Between γ–Ray Irradiation and 3DEG f...,/document/9724229/,2022,"['Khushwant Sehra', 'Vandana Kumari', 'Poonam ...",0,This work investigates the cumulative dose \n6...
1,132,132,Practical Methods for Efficient Resource Utili...,/document/9281038/,2020,"['George Koutitas', 'Shashwat Vyas', 'Chaitany...",0,This work presents a novel approach that adopt...
2,576,576,Application of Non-Negative Tensor Factorizati...,/document/8911489/,2019,"['Han Zhong', 'Geqi Qi', 'Wei Guan', 'Xiaochen...",1,With the rapid development of civil aviation t...


In [ ]:
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')

def text_preprocessing(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'@\w+\s*', ' ', text)
    text = re.sub(r'\\W',' ',text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'http', ' ', text)
    text = re.sub(r'<.*?>+', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', ' ', text)
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]', ' ', text)
    return text

def drop_stopwords(text):
    dropped = [word for word in text.split() if word not in stop_words]
    final_text = ' '.join(dropped)
    return final_text

def lemmatization(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

def delete_one_characters(text):
    deleted = [word if len(word)>1 else "" for word in text.split()]
    final_text = ' '.join(deleted)
    return final_text

data = data_raw.copy()
data['preprocessed_abstract'] = data['abstract'].apply(text_preprocessing).apply(drop_stopwords).apply(lemmatization).apply(delete_one_characters)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Id,title,link,year,authors,citations,abstract,preprocessed_abstract
0,261,261,Interplay Between γ–Ray Irradiation and 3DEG f...,/document/9724229/,2022,"['Khushwant Sehra', 'Vandana Kumari', 'Poonam ...",0,This work investigates the cumulative dose \n6...,work investigate cumulative dose co gamma gam...
1,132,132,Practical Methods for Efficient Resource Utili...,/document/9281038/,2020,"['George Koutitas', 'Shashwat Vyas', 'Chaitany...",0,This work presents a novel approach that adopt...,work present novel approach adopt content cach...
2,576,576,Application of Non-Negative Tensor Factorizati...,/document/8911489/,2019,"['Han Zhong', 'Geqi Qi', 'Wei Guan', 'Xiaochen...",1,With the rapid development of civil aviation t...,rapid development civil aviation transportatio...
3,454,454,Novel Method for Magnetic Flux Density Estimat...,/document/9705564/,2022,"['Adnan Mujezinovic', 'Emir Turajlic', 'Ajdin ...",0,"In this paper, a novel method for the magnetic...",paper novel method magnetic flux density estim...
4,337,337,Separation Behaviour Difference Between Gelati...,/document/8918179/,2019,"['Chao Cao', 'Jiyun Zhao', 'Guilin Li', 'Haiga...",2,Different from the rigid separation of biologi...,different rigid separation biological tissue s...


In [ ]:
text_corpus = data['preprocessed_abstract'].values
nested_document_tokens  = [t.split() for t in text_corpus]

# dictionary
id2word = Dictionary(nested_document_tokens)
id2word.filter_extremes(no_below=15, no_above=0.8)

# corpus
corpus = [id2word.doc2bow(text) for text in nested_document_tokens]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_TOPICS = 6

lda_model = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=id2word, iterations = 300, random_state = SEED, passes=15)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pprint.pprint(lda_model.print_topics())

[(0,
  '0.038*"image" + 0.025*"feature" + 0.017*"use" + 0.017*"method" + '
  '0.015*"propose" + 0.013*"detection" + 0.010*"result" + 0.010*"base" + '
  '0.009*"performance" + 0.009*"model"'),
 (1,
  '0.021*"method" + 0.021*"propose" + 0.020*"system" + 0.017*"base" + '
  '0.015*"control" + 0.013*"use" + 0.010*"approach" + 0.009*"user" + '
  '0.008*"study" + 0.008*"set"'),
 (2,
  '0.026*"algorithm" + 0.020*"propose" + 0.019*"use" + 0.016*"base" + '
  '0.016*"method" + 0.015*"network" + 0.014*"problem" + 0.012*"system" + '
  '0.012*"optimization" + 0.011*"datum"'),
 (3,
  '0.047*"model" + 0.020*"propose" + 0.020*"network" + 0.019*"use" + '
  '0.018*"datum" + 0.016*"method" + 0.015*"base" + 0.014*"prediction" + '
  '0.013*"result" + 0.012*"performance"'),
 (4,
  '0.024*"power" + 0.016*"high" + 0.016*"voltage" + 0.013*"result" + '
  '0.013*"use" + 0.012*"antenna" + 0.011*"frequency" + 0.011*"propose" + '
  '0.011*"system" + 0.011*"low"'),
 (5,
  '0.022*"propose" + 0.022*"signal" + 0.015*"sy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import os
import pickle
import pyLDAvis
import pyLDAvis.gensim

NUM_TOPICS = 6
# ... (your existing code)

pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/kaggle/working/', 'ldavis.html')  # Make sure directory exists

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(LDAvis_data_filepath), exist_ok=True)

if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/kaggle/working/ldavis.html')
LDAvis_prepared

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.035112  0.041420       1        1  21.906426
3      0.068377 -0.044333       2        1  17.235916
0      0.076692 -0.092465       3        1  16.965162
5     -0.042690  0.049272       4        1  15.896964
1      0.043917  0.084925       5        1  14.453905
4     -0.181408 -0.038819       6        1  13.541627, topic_info=        Term        Freq       Total Category  logprob  loglift
60     image  319.000000  319.000000  Default  30.0000  30.0000
171    model  584.000000  584.000000  Default  29.0000  29.0000
331  feature  275.000000  275.000000  Default  28.0000  28.0000
294   signal  187.000000  187.000000  Default  27.0000  27.0000
200    power  268.000000  268.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
81   propose   69.688356  844.375658   Topic6  -4.4921  -0.4952
180      two   47.220099  237.234547   Topic6  -4.8813   0.3852
174    paper   49.004648  339.457239   Topic6  -4.8442   0.0640
140     show   43.846993  276.529866   Topic6  -4.9554   0.1578
45      base   41.359392  599.143164   Topic6  -5.0138  -0.6738

[365 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
464       1  0.168674   access
464       2  0.253011   access
464       3  0.042169   access
464       4  0.484938   access
464       5  0.063253   access
...     ...       ...      ...
182       4  0.136262     well
182       5  0.118489     well
182       6  0.154035     well
406       2  0.119183  whether
406       5  0.774688  whether

[1171 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 6, 2, 5])

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=nested_document_tokens, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print("Coherence Score \t:{0:.7f}".format(coherence_lda))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence Score 	:0.3914806


1. I applied several preprocessing operations (cleaning,dropping stop words, lemmatizing),
2. I have chosen 6 for the number of topics. (I tried the topics number value from 5 to 30, but there was no betterment in respect of the Coherence Score.)
3. Topic modelling visualization with pyLDAvis library [2].
4. An LDA model evaluation with the Coherence Score.